In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata as ad
import scanpy as sc
import os
import squidpy as sq
from sklearn.preprocessing import LabelEncoder
import json

import torch
import torchsde
from torchdyn.core import NeuralODE
from tqdm import tqdm
import sys
sys.path.append('/Users/rssantanu/Desktop/codebase/constrained_FM')
from src.ode_helper_functions import *
from src.data_loading import preprocess_data, process_data, get_batch
from src.plots import plot_trajectories, plot_trajectories_new
from src.evaluate import *

from torchcfm.conditional_flow_matching import *
from torchcfm.models import MLP, GradModel, Graph_like_transformer
from torchcfm.utils import torch_wrapper

import argparse

MODEL_REGISTRY = {
    "mlp": MLP,
    "graph_like_transformer": Graph_like_transformer,
    "gradmodel": GradModel,
    # Add more models here as needed
}


In [46]:
scRNA = ad.read_h5ad(os.path.join('/Users/rssantanu/Desktop/codebase/constrained_FM', 'datasets/h5ad_processed_datasets', 'GSE232025_stereoseq.h5ad'))
labels_dict = {'1':1, '2':2, '3':3, '4':4, '5':5}
cell_type_key = 'celltype'

params = json.load(open('/Users/rssantanu/Desktop/codebase/constrained_FM/train_configs/v7_pca.json'))

# add more params keys
params['dim'] = scRNA.obsm[f'X_{params["representation"]}'].shape[1]
params['out_dim'] = scRNA.obsm[f'X_{params["representation"]}'].shape[1]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

day_list = (scRNA.obs["day"]).values.tolist()

X_raw, data_df = preprocess_data(scRNA)
X_phate, X_phate_conditional, Spatial, Celltype_list = process_data(scRNA, X_raw, cell_type_key, labels_dict, params['use_spatial'], params['use_celltype'], params['use_bio_prior'], params['representation'])


/Users/rssantanu/anaconda3/envs/torch_flow_matching/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [44]:
from sklearn.model_selection import StratifiedKFold, train_test_split
import numpy as np

all_data = [0,1,2,3,4]

a1,b1 = train_test_split(all_data[1:], test_size=0.2, random_state=42)
a2,b2 = train_test_split(all_data[1:], test_size=0.2, random_state=43)
a3,b3 = train_test_split(all_data[1:], test_size=0.2, random_state=44)
a4,b4 = train_test_split(all_data[1:], test_size=0.2, random_state=45)


# n_splits = 5
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# X_phate = np.array(X_phate)
# X_phate_conditional = np.array(X_phate_conditional)
# Spatial = np.array(Spatial)
# Celltype_list = np.array(Celltype_list)

# for fold, (train_idx, test_idx) in enumerate(skf.split(X_phate, [1]*len(X_phate))):
#     print(train_idx)
#     print(test_idx)

    # Xp_train, Xp_test = X_phate[train_idx], X_phate[test_idx]
    # Xpc_train, Xpc_test = X_phate_conditional[train_idx], X_phate_conditional[test_idx]
    # S_train, S_test = Spatial[train_idx], Spatial[test_idx]
    # C_train, C_test = Celltype_list[train_idx], Celltype_list[test_idx]
    # print(f"Fold {fold+1}: Train size {len(train_idx)}, Test size {len(test_idx)}")

In [45]:
np.arange(len(all_data))

array([0, 1, 2, 3, 4])

In [43]:
print(sorted(a1), sorted(a2), sorted(a3), sorted(a4))

[1, 3, 4] [1, 2, 4] [1, 2, 3] [2, 3, 4]


In [40]:
all_data[[3,4,5]]

TypeError: list indices must be integers or slices, not list